In [53]:
import numpy as np

from typing import Callable

# Encoding.

In [54]:
def binary_encode(val: float) -> list[int]:
  precision: int = 4;

  # Separate int and float parts
  int_part: int = int(val);
  float_part: float = val - int_part;

  # Convert int part to binary
  bin_int: str = bin(int_part)[2:];

  # Convert float part to binary using 4-digits of precision
  bin_float: str = "";
  while precision > 0:
    float_part *= 2;
    bit: int = int(float_part);
    bin_float += str(bit);
    float_part -= bit;
    precision -= 1;
  
  # Combine int and float parts into final array
  bin_val: list[int] = [int(bit) for bit in bin_int + bin_float];

  # Handle negative numbers
  if val < 0:
    bin_val.insert(0, 1)
  else:
    bin_val.insert(0, 0)

  return bin_val;

In [55]:
def binary_decode(bin_arr: list[int]) -> float:
  precision: int = 4;
  sign: int = -1 if bin_arr.pop(0) == 1 else 1;
  
  # Split binary array into its int and float parts
  int_size: int = len(bin_arr) - precision;
  bin_int: list[int] = bin_arr[:int_size];
  bin_float: list[int] = bin_arr[int_size:];

  # Convert int part to decimal
  int_part: int = 0;
  for i, bit in enumerate(reversed(bin_int)):
    int_part += bit * (2 ** i);
  
  # Convert float part to decimal
  float_part: float = 0.0;
  for i, bit in enumerate(bin_float):
    float_part += bit * (2 ** -(i + 1));
  
  # Combine int and float parts
  dec_val: float = sign * (int_part + float_part);

  return dec_val;

In [56]:
# Test binary encoding and decoding
dec_val: float = 23.6875;
print(f'Original value is {dec_val}');

bin_rep: list[int] = binary_encode(dec_val);
print(f'Value after binary encoding: {bin_rep}');

decoded_bin_rep: float = binary_decode(bin_rep);
print(f'Value after decoding binary representation: {decoded_bin_rep}');

Original value is 23.6875
Value after binary encoding: [0, 1, 0, 1, 1, 1, 1, 0, 1, 1]
Value after decoding binary representation: 23.6875


In [43]:
def real_encode(val: float) -> list[int]:
  precision: int = 4;

  # Round decimal value to 4-digit precision
  round_val: float = round(val, precision);

  # Scale value to an int
  scaled_val: int = int(round_val * (10 ** precision));

  # Convert into real representation
  real_rep: list[int] = [int(digit) for digit in str(abs(scaled_val))];

  # Handle negative numbers
  if val < 0:
    real_rep.insert(0, 1)
  else:
    real_rep.insert(0, 0)

  return real_rep;

In [41]:
def real_decode(real_arr: list[int]) -> float:
  precision: int = 4;
  sign: int = -1 if real_arr.pop(0) == 1 else 1;

  # Convert array into an int
  scaled_int: int = int(''.join(map(str, real_arr)));

  # Scale down to the original float
  dec_val: float = sign * (scaled_int / (10 ** precision))

  return dec_val;

In [45]:
# Test real encoding and decoding
dec_val: float = -dec_val
print(f'Original value is {dec_val}');

real_rep: list[int] = real_encode(dec_val);
print(f'Value after real encoding: {real_rep}');

decoded_real_rep: float = real_decode(real_rep);
print(f'Value after decoding real representation: {decoded_real_rep}');

Original value is -23.6875
Value after real encoding: [1, 2, 3, 6, 8, 7, 5]
Value after decoding real representation: -23.6875


# Crossover.

# Mutation.

# Selection.

In [48]:
def roulette_wheel_selection(
    population: list[list[list[int]]], 
    func: Callable[[list[list[int]]], float]
  ) -> tuple[list[list[int]], float]:
  # Compute fitness for each individual in the population
  fitness_arr: list[float] = [func(individual) for individual in population];

  # Compute cumulated fitness of the population
  S: float = np.sum(fitness_arr);

  # Compute probability and cumulated probability for each individual
  probability_arr: list[float] = [fitness / S for fitness in fitness_arr];
  cum_probability_arr: list[float] = np.cumsum(probability_arr);

  # Obtain a random number in [0, 1]
  r: float = np.random.random();

  # Select the individual
  for i, cum_probability in enumerate(cum_probability_arr):
    if r <= cum_probability:
      return population[i], func(population[i]);

  # Return last individual if rounding error
  return population[-1], func(population[-1]);

In [49]:
def binary_tournament_selection(
        population: list[list[list[int]]], 
        func: Callable[[list[list[int]]], float]
    ) -> tuple[list[list[int]], float]:
    # Perform tournament until one winner remains
    while len(population) > 1:
        winner_arr = []

        # Perform selection in pairs
        while len(population) > 1:
            # Choose two individuals randomly without replacement
            first_individual = np.random.choice(population)
            population.remove(first_individual)

            second_individual = np.random.choice(population)
            population.remove(second_individual)

            # Compare their fitness
            if func(first_individual) > func(second_individual):
                winner_arr.append(first_individual)
            else:
                winner_arr.append(second_individual)

        # If odd number of individuals, advance the last individual automatically
        if len(population) == 1:
            winner_arr.append(population[0])
        
        # Set the population for the next round
        population = winner_arr

    # Return the last remaining winner
    return population[0], func(population[0]);

# Genetic Algorithm.

In [57]:
def simple_GA(
    func: Callable[[list[list[int]]], float], 
    n: int,
    up_bound: float, 
    low_bound: float, 
    encode_func: Callable[[float], list[int]], 
    decode_func: Callable[[list[int]], float], 
    selec_func: Callable[[list[list[list[int]]], Callable[[list[list[int]]], float]], tuple[list[list[int]], float]], 
    mu: int = 10,
    fit_threshold: float = 0.001,
    max_generations: int = 10000
  ) -> list[float]:
  generations: int = 0;

  # Create initial population with mu individuals
  initial_population: list[list[float]] = [];

  for _ in range(mu):
    # Generate n random numbers for initialization
    random_numbers: list[float] = np.random.uniform(low_bound, up_bound, n);
    initial_population.append(random_numbers);
  
  # Encode the numbers
  encoded_reps: list[list[list[int]]] = [[encode_func(number) for number in individual] for individual in initial_population]

  # Array to track last five generations fitness
  gen_hist_fit: list[float] = [];
  
  while generations < max_generations:
    generations += 1;

    # Decode the representation
    decoded_reps: list[list[float]] = [[decode_func(rep) for rep in individual] for individual in encoded_reps]

    # Select two parents usign a selection mehcanism
    first_parent, first_fit = selec_func(decoded_reps, func);
    print(f'First selected parent is: {first_parent}');
    decoded_reps.remove(first_parent);

    second_parent, second_fit = selec_func(decoded_reps, func);
    print(f'Second selected parent is: {second_parent}');
    decoded_reps.remove(second_parent);

    if len(gen_hist_fit) == 5:
      # Replace fitness history if array exceeds 5 generations
      gen_hist_fit.pop(0);

      # Check for significant changes over the last 5 generations
      delta: float = np.max(gen_hist_fit) - np.min(gen_hist_fit);
      if delta < fit_threshold:
        print(f'Stopping as changes in the last 5 generations are too small: {delta}');
        break

    gen_hist_fit.append(np.mean([first_fit, second_fit]));

    # TODO: Add crossover and mutation operations to generate a child
    # Placeholder for mutation and crossover operations
    child = first_parent  # Replace this with actual mutation and crossover logic
        
    # Encode the child again and replace the least fit individual
    encoded_child = [encode_func(num) for num in child]
    encoded_reps.append(encoded_child)  # Add the new child to the population

    # Remove the least fit individual if necessary (this can be done using the fitness function)
    if len(encoded_reps) > mu:
        # Find and remove the least fit individual
        least_fit_index = np.argmin([func(decode_func(rep)) for rep in encoded_reps])
        del encoded_reps[least_fit_index]
  
  # Select fitest element after genetic process ends
  decoded_reps: list[float] = [decode_func(rep) for rep in encoded_reps];
  best_element: list[list[int]] = selec_func(decoded_reps, func);
  print(f'Best element is: {best_element}, completed in {generations} generations.');

  return best_element;